In [135]:
import os 
import logging 
import gzip 
import numpy as np 
import pandas as pd
import json 

In [136]:
def ungzip(file_path):
    """
    Take a file path and ungzip it
    """
    # TODO: there should probably be some error checks here
    ungzipped_file_path = file_path.rstrip('.gz')
    with gzip.open(file_path, 'rb') as gz_file:
        file_content = gz_file.read()
    return file_content.decode('utf-8')


#dns log 
{
  "ts": 1591367999.306059,
  "uid": "CMdzit1AMNsmfAIiQc",
  "id.orig_h": "192.168.4.76",
  "id.orig_p": 36844,
  "id.resp_h": "192.168.4.1",
  "id.resp_p": 53,
  "proto": "udp",
  "trans_id": 8555,
  "query": "testmyids.com",
  "qclass": 1,
  "qclass_name": "C_INTERNET",
  "qtype": 28,
  "qtype_name": "AAAA",
  "rcode": 0,
  "rcode_name": "NOERROR",
  "AA": false,
  "TC": false,
  "RD": true,
  "RA": false,
  "Z": 0,
  "rejected": false
}
https://docs.zeek.org/en/master/logs/dns.html 

## conn.log

In [137]:
current_dir_path = '/usr/local/logs/2024-02-12'
if not os.path.islink(current_dir_path):
    # sub_dir is now any given historical data directory 
    logging.info(f"Checking {current_dir_path}")
    for file in os.listdir(current_dir_path):
        # file is now any given file in the historical data directory
        current_file_path = os.path.join(current_dir_path, file)
        if "conn." in file: #conn.
            # get the whole file in memory
            logging.info(f"Opening file {current_file_path}")
            json_data_file = ungzip(current_file_path)

In [138]:
import json 
for line in json_data_file.splitlines():
        # log_entry is now a single json log from the file
        log_entry = json.loads(line.strip())
        print(log_entry)

{'ts': 1707757823.364103, 'uid': 'C0LEGs2p93lnNEFB5f', 'id.orig_h': '192.168.0.168', 'id.orig_p': 65363, 'id.resp_h': '192.168.0.1', 'id.resp_p': 53, 'proto': 'udp', 'service': 'dns', 'duration': 0.019865989685058594, 'orig_bytes': 34, 'resp_bytes': 141, 'conn_state': 'SF', 'local_orig': True, 'local_resp': True, 'missed_bytes': 0, 'history': 'Dd', 'orig_pkts': 1, 'orig_ip_bytes': 62, 'resp_pkts': 1, 'resp_ip_bytes': 169}
{'ts': 1707757823.364033, 'uid': 'CoIaps3LBPANWZX887', 'id.orig_h': '192.168.0.168', 'id.orig_p': 51657, 'id.resp_h': '192.168.0.1', 'id.resp_p': 53, 'proto': 'udp', 'service': 'dns', 'duration': 0.012295961380004883, 'orig_bytes': 34, 'resp_bytes': 114, 'conn_state': 'SF', 'local_orig': True, 'local_resp': True, 'missed_bytes': 0, 'history': 'Dd', 'orig_pkts': 1, 'orig_ip_bytes': 62, 'resp_pkts': 1, 'resp_ip_bytes': 142}
{'ts': 1707757828.897382, 'uid': 'CllN3R2OE84qgP4Myl', 'id.orig_h': '192.168.0.168', 'id.orig_p': 51400, 'id.resp_h': '142.250.80.74', 'id.resp_p': 

In [139]:
features = ["id.orig_h", "id.resp_h", "proto", "service", "duration", "conn_state", "local_orig","local_resp",
            "missed_bytes","history", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]
data_list = []
for line in json_data_file.splitlines():
    # log_entry is now a single json log from the file
    log_entry = json.loads(line.strip())
    
    # Check if each feature is present in the log_entry
    feature_values = [log_entry.get(feature, None) for feature in features]
    data_list.append(feature_values)

df = pd.DataFrame(data_list, columns=features)

In [140]:
df

,id.orig_h,id.resp_h,proto,service,duration,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes
0,192.168.0.168,192.168.0.1,udp,dns,0.019866,SF,True,True,0,Dd,1,62,1,169
1,192.168.0.168,192.168.0.1,udp,dns,0.012296,SF,True,True,0,Dd,1,62,1,142
2,192.168.0.168,142.250.80.74,tcp,None,0.016519,RSTR,True,False,0,DFr,2,128,2,80
3,192.168.0.168,224.0.0.251,udp,dns,7.450594,S0,True,False,0,D,2,469,0,0
4,fe80::17:2915:d910:f37,ff02::fb,udp,dns,7.450569,S0,True,False,0,D,2,509,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,192.168.0.168,20.189.172.32,tcp,ssl,0.347173,SF,True,False,0,ShADdfFa,12,1921,9,5408
61,192.168.0.168,20.189.172.32,tcp,ssl,0.332572,SF,True,False,0,ShADdfFa,12,1921,9,5408
62,192.168.0.168,20.189.172.32,tcp,ssl,0.338312,SF,True,False,0,ShADdfFa,12,1921,9,5408
63,192.168.0.168,20.189.172.32,tcp,ssl,0.343127,SF,True,False,0,ShADdfFa,12,2484,9,5408


In [141]:
log_entry.keys()

dict_keys(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 'proto', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes'])

#### utils

In [38]:
#utils 
import ipaddress
def one_hot_encode(df, column_name):
    for col in column_name:
        if col in df.columns:
            df = pd.get_dummies(data=df, columns=[col])
    return df

def create_broadcast_variable(new_df):
    # create broadcast variable
    # can have more than one broadcast address
    #255 is the broadcast address for ipv4 
    if 'id.resp_h' in new_df.columns:
        new_df['is_destination_broadcast'] = new_df['id.resp_h'].apply(lambda x: 1 if "255" in x[-3:] else 0) 
    return new_df

def create_direction_variable(new_df):
    # #create traffic direction variable
    new_df['traffic_direction']        = new_df.apply(lambda x: get_traffic_direction(x['id.orig_h'], x['id.resp_h']), axis=1) 
    return new_df


def get_traffic_direction(source_ip, destination_ip):
    """
    Takes a source and destination IP address and returns the direction of the traffic.
    Please ensure the source and destination are correct as this is useless without the verification of the parameters.

    Parameters
    ----------
    source_ip: str
        Source IP address of the flow.
    destination_ip: str
        Destination IP address of the flow.
    
    Returns
    -------
    str: string indicating the direction. Can be 'internal', 'outgoing', 'incoming' or 'external'.
    """
    src_ip = ipaddress.ip_address(source_ip) 
    dest_ip = ipaddress.ip_address(destination_ip) 
    if src_ip.version == 6 or dest_ip.version ==6:
        return "IPv6"
    
    if is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "internal"
    elif is_private_ip(source_ip) and not is_private_ip(destination_ip):
        return "outgoing"
    elif not is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "incoming"
    else:
        return "external"    


def is_private_ip(ip_str):
    """
    Takes an IP string and returns whether the IP is private or not per RFC 1918.

    Parameters
    ----------
    ip_str: str
        String of an IP address.

    Returns
    -------
    bool: a bool of whether or not the IP is private.
    """
    try:
        ip = ipaddress.ip_address(ip_str)
        if ip.version == 4:
            return ip.is_private
        else:
            return False  # Ignore IPv6 addresses
    except ValueError:
        return False  # Invalid IP address format

def makedf_samecol(cols, new_df):
    #Create these columns if they are not present in the original df and fill them with 0s. 
    # Ensure that all the specified columns are present even if they are not present in the original df. 

    for col in cols:
        if col not in new_df.columns:
            new_df[col] = 0
    return new_df[cols]

def create_history_variable(new_df):
    # break out history variable
    
    if 'history' not in new_df.columns: 
        new_df['history'] = 'N'  

    #fill NaNs with 'N'
    new_df['history'] = new_df['history'].fillna('N') 
    new_df['history_has_S'] = new_df['history'].apply(lambda x: 1 if "S" in x else 0)
    new_df['history_has_h'] = new_df['history'].apply(lambda x: 1 if "h" in x else 0)
    new_df['history_has_A'] = new_df['history'].apply(lambda x: 1 if "A" in x else 0)
    new_df['history_has_D'] = new_df['history'].apply(lambda x: 1 if "D" in x else 0)
    new_df['history_has_a'] = new_df['history'].apply(lambda x: 1 if "a" in x else 0)
    new_df['history_has_d'] = new_df['history'].apply(lambda x: 1 if "d" in x else 0)
    new_df['history_has_F'] = new_df['history'].apply(lambda x: 1 if "F" in x else 0)
    new_df['history_has_f'] = new_df['history'].apply(lambda x: 1 if "f" in x else 0)
    new_df['history_has_N'] = new_df['history'].apply(lambda x: 1 if "N" in x else 0)
    new_df = new_df.drop(columns='history')
    return new_df 

def fill_na(new_df):
    
    #Fill Nans with 0s : duration, orig_bytes resp_bytes
    # Specify the columns you want to fill with zeros
    columns_to_fill_with_zeros = ['duration', 'orig_bytes', 'resp_bytes']
    # Check if columns exist; if not, create and fill with zeros
    for col in columns_to_fill_with_zeros:
        if col not in new_df.columns:
            new_df[col] = 0
    new_df[columns_to_fill_with_zeros] = new_df[columns_to_fill_with_zeros].fillna(0)
    
    #Fill Nans with 'Other' : service
    columns_to_fill_with_other = ['service']
    if 'service' in new_df.columns:
    # new_df['service'].fillna('other', inplace=True)
        new_df['service'] = new_df['service'].fillna('other')
        
    return new_df

#### Get raw data for Olive

In [39]:
def get_raw_conn(json_data_file):
    features = ["id.orig_h", "id.resp_h", "proto", "service", "duration", "conn_state", "local_orig","local_resp",
            "missed_bytes","history", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]
    data_list = []
    for line in json_data_file.splitlines():
        # log_entry is now a single json log from the file
        log_entry = json.loads(line.strip())
        
        # Check if each feature is present in the log_entry
        feature_values = [log_entry.get(feature, None) for feature in features]
        data_list.append(feature_values)

    df = pd.DataFrame(data_list, columns=features)
    df = create_broadcast_variable(df)
    df = create_direction_variable(df)
    # Convert the boolean values in columns "local_orig" and "local_resp" to 1 and 0s
    df['local_orig'] = df['local_orig'].astype(int)
    df['local_resp'] = df['local_resp'].astype(int)
    
    return df 

In [40]:
df = get_raw_conn(json_data_file)

In [41]:
df

,id.orig_h,id.resp_h,proto,service,duration,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,is_destination_broadcast,traffic_direction
0,192.168.0.168,192.168.0.1,udp,dns,0.019866,SF,1,1,0,Dd,1,62,1,169,0,internal
1,192.168.0.168,192.168.0.1,udp,dns,0.012296,SF,1,1,0,Dd,1,62,1,142,0,internal
2,192.168.0.168,142.250.80.74,tcp,None,0.016519,RSTR,1,0,0,DFr,2,128,2,80,0,outgoing
3,192.168.0.168,224.0.0.251,udp,dns,7.450594,S0,1,0,0,D,2,469,0,0,0,outgoing
4,fe80::17:2915:d910:f37,ff02::fb,udp,dns,7.450569,S0,1,0,0,D,2,509,0,0,0,IPv6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,192.168.0.168,20.189.172.32,tcp,ssl,0.347173,SF,1,0,0,ShADdfFa,12,1921,9,5408,0,outgoing
61,192.168.0.168,20.189.172.32,tcp,ssl,0.332572,SF,1,0,0,ShADdfFa,12,1921,9,5408,0,outgoing
62,192.168.0.168,20.189.172.32,tcp,ssl,0.338312,SF,1,0,0,ShADdfFa,12,1921,9,5408,0,outgoing
63,192.168.0.168,20.189.172.32,tcp,ssl,0.343127,SF,1,0,0,ShADdfFa,12,2484,9,5408,0,outgoing


In [42]:
#This code checks for null values in each feature
has_null = []
for feature in df.columns:
    null_count = df[feature].isnull().sum()
    print(f"Null count for {feature}: {null_count}")    
    if null_count:
        has_null.append(feature)

print("\nhas_null",has_null) # has_null ['host'] 

# Create a variable for each feature that contains null, with the column name "has_null_featurename"
for feature in has_null: 
    df[f'has_{feature}'] = df[feature].notnull().astype(int)


Null count for id.orig_h: 0
Null count for id.resp_h: 0
Null count for proto: 0
Null count for service: 17
Null count for duration: 6
Null count for conn_state: 0
Null count for local_orig: 0
Null count for local_resp: 0
Null count for missed_bytes: 0
Null count for history: 3
Null count for orig_pkts: 0
Null count for orig_ip_bytes: 0
Null count for resp_pkts: 0
Null count for resp_ip_bytes: 0
Null count for is_destination_broadcast: 0
Null count for traffic_direction: 0

has_null ['service', 'duration', 'history']


In [43]:
has_null = ['service']
# Create a variable to track if the feature contains null. Create a column "has_null_featurename"
for feature in has_null: 
    df[f'has_{feature}'] = df[feature].notnull().astype(int)
df['service'] = df['service'].replace({False: 0, True: 1})

In [44]:
df[['service','has_service']]

,service,has_service
0,dns,1
1,dns,1
2,None,0
3,dns,1
4,dns,1
...,...,...
60,ssl,1
61,ssl,1
62,ssl,1
63,ssl,1


### Get data for conn for kitnet (w/o aggregation)


In [533]:
def preprocess_json_conn(json_batch):
    """
    This function receives a json batch from the main control flow of the train 
    functions. It should convert the conn.log of the json_batch to a numpy 2D array, apply necessary transformations,
    then return it. 

    Note: the input is only one unzipped json file. 
    """
    features = ["id.orig_h", "id.resp_h", "proto", "service", "duration", "conn_state", 
                "local_orig","local_resp","missed_bytes","history", 
                "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]
    #TODO: add features: duration, local_orig, local_resp 
    data_list = []
    for line in json_batch.splitlines():
        # log_entry is now a single json log from the file 
        log_entry = json.loads(line.strip())
        # data_list.append([log_entry[feature] for feature in features])
        # Check if each feature is present in the log_entry
        feature_values = [log_entry.get(feature, None) for feature in features]
        data_list.append(feature_values)
    #Re-use the preprocess function from last sem by Zoe. 
    #TODO: optimize the code via removing pandas
    new_df = pd.DataFrame(data_list, columns=features) 
    #Fill NaNs with 0s : duration, orig_bytes resp_bytes, if there are no columns, create one and fill with 0s 
    new_df = fill_na(new_df) 
    # create history, broadcast, traffic_direction variables
    new_df = create_history_variable(new_df)
    new_df = create_broadcast_variable(new_df)
    new_df = create_direction_variable(new_df)
    # one hot encode categorical variables
    column_name = ['conn_state', "proto", "traffic_direction" , "service"]
    new_df = one_hot_encode(new_df, column_name)
    # Convert the boolean values in columns "local_orig" and "local_resp" to 1 and 0s
    new_df['local_orig'] = new_df['local_orig'].astype(int)
    new_df['local_resp'] = new_df['local_resp'].astype(int)
    # make sure the columns are the same as the original df
    #TODO: to be confirmed once HSRN EDA is done
    cols = ['conn_state_OTH', 'conn_state_REJ','conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR','conn_state_RSTRH', 
        'conn_state_S0', 'conn_state_S1', 'conn_state_S2','conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR',
        'proto_tcp', 'proto_udp', 
        'service_dhcp', 'service_dns','service_http', 'service_irc','service_ntp',
        'service_other', 'service_ssh','service_ssl',
        'traffic_direction_external','traffic_direction_incoming', 
        'traffic_direction_internal','traffic_direction_outgoing',
        "duration","local_orig","local_resp","missed_bytes","orig_pkts","orig_ip_bytes","resp_pkts","resp_ip_bytes"]
    new_df = makedf_samecol(cols, new_df)
    # Convert DataFrame to NumPy array
    np_arr = new_df.to_numpy(dtype=np.float32)
    return np_arr

In [534]:
preprocess_json_conn(json_data_file)

array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 6.200e+01, 1.000e+00,
        1.690e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 6.200e+01, 1.000e+00,
        1.420e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.280e+02, 2.000e+00,
        8.000e+01],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.921e+03, 9.000e+00,
        5.408e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.484e+03, 9.000e+00,
        5.408e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 4.000e+01, 1.000e+00,
        4.000e+01]], dtype=float32)

### Get data for conn for kitnet (w aggregation)


In [488]:
id_feature = ["id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p"]
features = ["ts","uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p",
            "proto", "service", "duration", "conn_state", "local_orig","local_resp",
            "missed_bytes","history", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]
data_list = []
for line in json_data_file.splitlines():
    # log_entry is now a single json log from the file
    log_entry = json.loads(line.strip())
    
    # Check if each feature is present in the log_entry
    feature_values = [log_entry.get(feature, None) for feature in features]
    data_list.append(feature_values)

df = pd.DataFrame(data_list, columns=features)

In [489]:
#fill Nans with 0s : duration, orig_bytes resp_bytes
df = fill_na(df)  

if 'history' not in df.columns: 
    df['history'] = 'N'  
#fill NaNs with 'N'
df['history'] = df['history'].fillna('N')
df = create_broadcast_variable(df)
df = create_direction_variable(df)
# Convert the boolean values in columns "local_orig" and "local_resp" to 1 and 0s
df['local_orig'] = df['local_orig'].astype(int)
df['local_resp'] = df['local_resp'].astype(int)


In [490]:
print(df.shape)

(65, 22)


In [491]:
df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,conn_state,...,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes,resp_bytes,is_destination_broadcast,traffic_direction
0,1.707758e+09,C0LEGs2p93lnNEFB5f,192.168.0.168,65363,192.168.0.1,53,udp,dns,0.019866,SF,...,0,Dd,1,62,1,169,0,0,0,internal
1,1.707758e+09,CoIaps3LBPANWZX887,192.168.0.168,51657,192.168.0.1,53,udp,dns,0.012296,SF,...,0,Dd,1,62,1,142,0,0,0,internal
2,1.707758e+09,CllN3R2OE84qgP4Myl,192.168.0.168,51400,142.250.80.74,443,tcp,other,0.016519,RSTR,...,0,DFr,2,128,2,80,0,0,0,outgoing
3,1.707758e+09,CnAcZTvINoaJSPmC2,192.168.0.168,5353,224.0.0.251,5353,udp,dns,7.450594,S0,...,0,D,2,469,0,0,0,0,0,outgoing
4,1.707758e+09,CrKvqoo2pkmR8IcKf,fe80::17:2915:d910:f37,5353,ff02::fb,5353,udp,dns,7.450569,S0,...,0,D,2,509,0,0,0,0,0,IPv6


In [149]:
# Group the DataFrame by the specified columns
grouped_data = df.groupby(['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p'])

# Extract the first 'ts' value for each group (assuming timestamps are ordered)
first_ts = grouped_data['duration'].first()

# This will be a Series with the first 'ts' value for each unique combination of grouping keys
print(first_ts)

id.orig_h               id.orig_p  id.resp_h      id.resp_p
192.168.0.1             45892      192.168.0.255  20002        0.000000
                        46449      224.0.0.251    5353         0.000000
                        46818      224.0.0.251    5353         0.000000
                        58627      192.168.0.255  20002        0.000000
192.168.0.168           3          142.250.80.1   3            1.246119
                                                                 ...   
                        64633      17.248.175.21  443          0.017568
                        65363      192.168.0.1    53           0.019866
                        65528      142.250.80.1   443          0.119744
fe80::17:2915:d910:f37  143        ff02::16       0            0.000000
                        5353       ff02::fb       5353         7.450569
Name: duration, Length: 63, dtype: float64


In [150]:
first_ts[first_ts > 1]

id.orig_h               id.orig_p  id.resp_h      id.resp_p
192.168.0.168           3          142.250.80.1   3             1.246119
                        5353       224.0.0.251    5353          7.450594
                        51381      20.189.173.2   443          45.350865
                        51448      216.165.12.42  443          30.428033
                        59051      17.57.144.54   5223         49.082516
fe80::17:2915:d910:f37  5353       ff02::fb       5353          7.450569
Name: duration, dtype: float64

In [123]:
filtered_df = df[(df['id.orig_h'] == '192.168.0.168') & (df['id.orig_p'] == 51428)]

In [151]:
filtered_df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes,resp_bytes
31,1.707758e+09,ClKxU23lRlw5hdtGFj,192.168.0.168,51428,142.250.65.234,443,tcp,ssl,0.141091,S1,True,False,0,ShADad,10,1658,14,7626,0,0
42,1.707762e+09,CJRUNP37hqlOb5Poa5,192.168.0.168,51428,142.250.65.234,443,tcp,other,0.012353,RSTR,True,False,0,DFTr,4,397,1,40,0,0


In [251]:
(1.707762e+09 - 1.707758e+09)

4000.0

In [487]:
df.columns

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'conn_state', 'local_orig',
       'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes',
       'resp_pkts', 'resp_ip_bytes', 'orig_bytes', 'resp_bytes',
       'is_destination_broadcast', 'traffic_direction'],
      dtype='object')

In [310]:
#To be confirmed with HSRN data
windows = [1,10,60,360] #seconds 
grp = ['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p']
#TODO: aggregate the features to compute the windowed features 
windowed_features_num = ['mean','min','max','std','var','cnt', 'sum'] #for numerical features: duration, orig_pkts, orig_ip_bytes, resp_pkts, resp_ip_bytes
windowed_features_cat = ['nuniq','entropy'] #for categorical features: conn_state, proto, service, history
aggr_feature_num = ['duration, missed_bytes, orig_pkts, orig_ip_bytes, resp_pkts, resp_ip_bytes']
aggr_feature_num = ['local_orig', 'local_resp']

**Research on ways to do aggregation in time windows (Real-Time Aggregation Features )**
1. https://nussknacker.io/documentation/docs/1.1/scenarios_authoring/AggregatesInTimeWindows/ 
2. rolling mean


In [127]:
!pip install pandasql

  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=f55c3b6f2822eaa7fb687d2a2679ace938a115c3571028606f5828a4bf028d61
  Stored in directory: /Users/Zoe_1/Library/Caches/pip/wheels/63/e8/ec/75b1df467ecf57b6ececb32cb16f4e86697cbfe55cb0c51f07
Successfully built pandasql


### SQL

In [131]:
from pandasql import sqldf

In [186]:
output = sqldf('''SELECT * FROM df LIMIT 5''')

In [187]:
output

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,conn_state,...,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes,resp_bytes,is_destination_broadcast,traffic_direction
0,1.707758e+09,C0LEGs2p93lnNEFB5f,192.168.0.168,65363,192.168.0.1,53,udp,dns,0.019866,SF,...,0,Dd,1,62,1,169,0,0,0,internal
1,1.707758e+09,CoIaps3LBPANWZX887,192.168.0.168,51657,192.168.0.1,53,udp,dns,0.012296,SF,...,0,Dd,1,62,1,142,0,0,0,internal
2,1.707758e+09,CllN3R2OE84qgP4Myl,192.168.0.168,51400,142.250.80.74,443,tcp,other,0.016519,RSTR,...,0,DFr,2,128,2,80,0,0,0,outgoing
3,1.707758e+09,CnAcZTvINoaJSPmC2,192.168.0.168,5353,224.0.0.251,5353,udp,dns,7.450594,S0,...,0,D,2,469,0,0,0,0,0,outgoing
4,1.707758e+09,CrKvqoo2pkmR8IcKf,fe80::17:2915:d910:f37,5353,ff02::fb,5353,udp,dns,7.450569,S0,...,0,D,2,509,0,0,0,0,0,IPv6


In [216]:
df.dtypes


ts                          float64
uid                          object
id.orig_h                    object
id.orig_p                     int64
id.resp_h                    object
id.resp_p                     int64
proto                        object
service                      object
duration                    float64
conn_state                   object
local_orig                     bool
local_resp                     bool
missed_bytes                  int64
history                      object
orig_pkts                     int64
orig_ip_bytes                 int64
resp_pkts                     int64
resp_ip_bytes                 int64
orig_bytes                    int64
resp_bytes                    int64
is_destination_broadcast      int64
traffic_direction            object
dtype: object

In [188]:
output.columns

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'conn_state', 'local_orig',
       'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes',
       'resp_pkts', 'resp_ip_bytes', 'orig_bytes', 'resp_bytes',
       'is_destination_broadcast', 'traffic_direction'],
      dtype='object')

In [153]:
output.columns

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'conn_state', 'local_orig',
       'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes',
       'resp_pkts', 'resp_ip_bytes', 'orig_bytes', 'resp_bytes'],
      dtype='object')

In [214]:
query = '''
SELECT id.orig_h, id.orig_p, id.resp_h, id.resp_p, AVG(b.duration) as avg_duration
FROM df a 
JOIN df b
ON a.id.orig_h = b.id.orig_h AND a.id.orig_p = b.id.orig_p AND a.id.resp_h = b.id.resp_h AND a.id.resp_p = b.id.resp_p

WHERE a.ts - b.ts <= 60
GROUP BY a.id.orig_h, a.id.orig_p, a.id.resp_h, a.id.resp_p
'''

#calculate AVG(value) OVER (PARTITION BY group)
#df['mean_value'] = df.groupby('group').value.transform(np.mean)

In [222]:
df.dtypes

ts                          float64
uid                          object
id.orig_h                    object
id.orig_p                     int64
id.resp_h                    object
id.resp_p                     int64
proto                        object
service                      object
duration                    float64
conn_state                   object
local_orig                     bool
local_resp                     bool
missed_bytes                  int64
history                      object
orig_pkts                     int64
orig_ip_bytes                 int64
resp_pkts                     int64
resp_ip_bytes                 int64
orig_bytes                    int64
resp_bytes                    int64
is_destination_broadcast      int64
traffic_direction            object
dtype: object

In [221]:
import sqlite3
from sqlalchemy.types import VARCHAR, INTEGER, FLOAT
# cursor = conn.cursor()
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False) #Create an in-memory SQLite database.
df_dtypes = {'id.orig_p': 'INTEGER', 'id.resp_p': 'INTEGER'}
# df_dtypes = {'id.orig_h': 'VARCHAR', 'id.orig_p': 'INTEGER', 'id.resp_h': 'VARCHAR', 'id.resp_p': 'INTEGER'}
df.to_sql('df', engine, index=False, dtype=df_dtypes)
# df.to_sql('df', engine, index=False)  # Replace 'df' with your desired table name
result_df = pd.read_sql('SELECT  id.orig_p,id.resp_p FROM df', engine)
# result_df = pd.read_sql('SELECT id.orig_h, id.orig_p, id.resp_h, id.resp_p FROM df', engine)
# df.to_sql(name='df_sql', con=engine)



ValueError: The type of id.orig_p is not a SQLAlchemy type

In [211]:
result_df

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,conn_state,...,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes,resp_bytes,is_destination_broadcast,traffic_direction
0,1.707758e+09,C0LEGs2p93lnNEFB5f,192.168.0.168,65363,192.168.0.1,53,udp,dns,0.019866,SF,...,0,Dd,1,62,1,169,0,0,0,internal
1,1.707758e+09,CoIaps3LBPANWZX887,192.168.0.168,51657,192.168.0.1,53,udp,dns,0.012296,SF,...,0,Dd,1,62,1,142,0,0,0,internal
2,1.707758e+09,CllN3R2OE84qgP4Myl,192.168.0.168,51400,142.250.80.74,443,tcp,other,0.016519,RSTR,...,0,DFr,2,128,2,80,0,0,0,outgoing
3,1.707758e+09,CnAcZTvINoaJSPmC2,192.168.0.168,5353,224.0.0.251,5353,udp,dns,7.450594,S0,...,0,D,2,469,0,0,0,0,0,outgoing
4,1.707758e+09,CrKvqoo2pkmR8IcKf,fe80::17:2915:d910:f37,5353,ff02::fb,5353,udp,dns,7.450569,S0,...,0,D,2,509,0,0,0,0,0,IPv6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,1.707762e+09,CmsBId1mQSH21Nn8Xf,192.168.0.168,51473,20.189.172.32,443,tcp,ssl,0.347173,SF,...,0,ShADdfFa,12,1921,9,5408,0,0,0,outgoing
61,1.707762e+09,CPrXFE2XI3xFN2x4x6,192.168.0.168,51480,20.189.172.32,443,tcp,ssl,0.332572,SF,...,0,ShADdfFa,12,1921,9,5408,0,0,0,outgoing
62,1.707762e+09,CKX629PWRVQsJH3Df,192.168.0.168,51481,20.189.172.32,443,tcp,ssl,0.338312,SF,...,0,ShADdfFa,12,1921,9,5408,0,0,0,outgoing
63,1.707762e+09,CGzmAv3TE4THf4pQ5d,192.168.0.168,51482,20.189.172.32,443,tcp,ssl,0.343127,SF,...,0,ShADdfFa,12,2484,9,5408,0,0,0,outgoing


In [215]:
from sqlalchemy import text
with engine.connect() as conn:
   conn.execute(text(query)).fetchall()

OperationalError: (sqlite3.OperationalError) no such column: id.orig_h
[SQL: 
SELECT id.orig_h, id.orig_p, id.resp_h, id.resp_p, AVG(b.duration) as avg_duration
FROM df a 
JOIN df b
ON a.id.orig_h = b.id.orig_h AND a.id.orig_p = b.id.orig_p AND a.id.resp_h = b.id.resp_h AND a.id.resp_p = b.id.resp_p

WHERE a.ts - b.ts <= 60
GROUP BY a.id.orig_h, a.id.orig_p, a.id.resp_h, a.id.resp_p
]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [175]:
df.columns

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'conn_state', 'local_orig',
       'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes',
       'resp_pkts', 'resp_ip_bytes', 'orig_bytes', 'resp_bytes',
       'is_destination_broadcast', 'traffic_direction'],
      dtype='object')

### Pandas

#### EDA

In [435]:
#'ts' is unix format
#import 
from datetime import datetime

In [311]:
from datetime import datetime
print(datetime.fromtimestamp(1.707758e+09))

2024-02-12 12:13:20


In [314]:
df.shape

(65, 22)

In [282]:
# Knowing that there is only 1 group that has the same id.orig_h, id.orig_p, id.resp_h, id.resp_p
fil = df[(df['id.orig_h'] == '192.168.0.168') & (df['id.orig_p'] == 51428)]

In [283]:
fil

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,conn_state,...,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes,resp_bytes,is_destination_broadcast,traffic_direction,duration_mean
31,2024-02-12 17:09:42.990166016,ClKxU23lRlw5hdtGFj,192.168.0.168,51428,142.250.65.234,443,tcp,ssl,0.141091,S1,...,ShADad,10,1658,14,7626,0,0,0,outgoing,NaN
42,2024-02-12 18:12:59.069338112,CJRUNP37hqlOb5Poa5,192.168.0.168,51428,142.250.65.234,443,tcp,other,0.012353,RSTR,...,DFTr,4,397,1,40,0,0,0,outgoing,0.076722


In [315]:
import pandas as pd

# Define the timestamps
dt1 = pd.to_datetime('2024-02-12 17:09:42.990166016')
dt2 = pd.to_datetime('2024-02-12 18:12:59.069338112')

# Calculate the time difference
time_diff = dt2 - dt1

# Print the time difference in a human-readable format
print(time_diff)

# 01:03:16.079172096 
#compute the time difference into seconds 
print(time_diff.total_seconds())

0 days 01:03:16.079172096
3796.079172


#### Agg Function

In [498]:
windowed_features_num = ['mean','min','max','std','var','cnt','sum']
def calculate_agg_feature_num(df, agg_feature, window_size=5000):
  """
  This function adds a new column "{agg_feature}_{either mean, min, max, std, or var}" to the DataFrame.
  This column contains the aggregated features (mean/min/max/std/var/count/sum) of network flows within the past {window_size} seconds
  for each group with the same ['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p'].

  Args:
      df: The pandas DataFrame containing network flow data.
      window_size: Size of the window for calculating the average (default: 5000 seconds).

  Returns:
      A new DataFrame with the added aggregated feautre columns.
  """
  # Convert timestamp to datetime
  # df['ts'] = datetime.fromtimestamp(df['ts']) #assumes timestamps are in the local machine's timezone. not suggested  
  df['ts'] = pd.to_datetime(df['ts'], unit='s') 
  df = df.set_index('ts') 
  # Calculate the aggregated feature for each group
  # to avoid NaN values, calculate the population standard deviation, specified with std(ddof=0)
  grp = ['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p']
  df[f'{agg_feature}_mean_{window_size}'] = df.groupby(grp)[f'{agg_feature}'].transform(lambda x: x.rolling(f'{window_size}s', min_periods=1).mean())
  df[f'{agg_feature}_min_{window_size}'] = df.groupby(grp)[f'{agg_feature}'].transform(lambda x: x.rolling(f'{window_size}s', min_periods=1).min())
  df[f'{agg_feature}_max_{window_size}'] = df.groupby(grp)[f'{agg_feature}'].transform(lambda x: x.rolling(f'{window_size}s', min_periods=1).max())
  df[f'{agg_feature}_std_{window_size}'] = df.groupby(grp)[f'{agg_feature}'].transform(lambda x: x.rolling(f'{window_size}s', min_periods=1).std(ddof=0))
  df[f'{agg_feature}_var_{window_size}'] = df.groupby(grp)[f'{agg_feature}'].transform(lambda x: x.rolling(f'{window_size}s', min_periods=1).var(ddof=0))
  df[f'{agg_feature}_cnt_{window_size}'] = df.groupby(grp)[f'{agg_feature}'].transform(lambda x: x.rolling(f'{window_size}s', min_periods=1).count())
  df[f'{agg_feature}_sum_{window_size}'] = df.groupby(grp)[f'{agg_feature}'].transform(lambda x: x.rolling(f'{window_size}s', min_periods=1).sum())

  return df.reset_index()

result_df = calculate_agg_feature_num(df.copy(),agg_feature = 'duration')  # Apply function to a copy of df
print(result_df)


                              ts                 uid               id.orig_h  \
0  2024-02-12 17:10:23.364103168  C0LEGs2p93lnNEFB5f           192.168.0.168   
1  2024-02-12 17:10:23.364033024  CoIaps3LBPANWZX887           192.168.0.168   
2  2024-02-12 17:10:28.897382144  CllN3R2OE84qgP4Myl           192.168.0.168   
3  2024-02-12 17:10:23.682648832   CnAcZTvINoaJSPmC2           192.168.0.168   
4  2024-02-12 17:10:23.682689024   CrKvqoo2pkmR8IcKf  fe80::17:2915:d910:f37   
..                           ...                 ...                     ...   
60 2024-02-12 18:12:59.359659008  CmsBId1mQSH21Nn8Xf           192.168.0.168   
61 2024-02-12 18:12:59.394697984  CPrXFE2XI3xFN2x4x6           192.168.0.168   
62 2024-02-12 18:12:59.394788096   CKX629PWRVQsJH3Df           192.168.0.168   
63 2024-02-12 18:12:59.396743936  CGzmAv3TE4THf4pQ5d           192.168.0.168   
64 2024-02-12 18:13:01.153886976  CUN2d84adrzHQOxdJi           192.168.0.168   

    id.orig_p      id.resp_h  id.resp_p

In [499]:
df.columns

Index(['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p',
       'proto', 'service', 'duration', 'conn_state', 'local_orig',
       'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes',
       'resp_pkts', 'resp_ip_bytes', 'orig_bytes', 'resp_bytes',
       'is_destination_broadcast', 'traffic_direction'],
      dtype='object')

In [493]:
#For feature such as  local_orig , port,... numerical but can be treated as categorical
from scipy.stats import entropy
def calculate_agg_feature_cat(df, agg_feature, window_size=5000):
  """
  This function adds a new column "{agg_feature}_{either nunique or entropy}" to the DataFrame.
  This column contains the aggregated features (nunique/entropy) of network flows within the past {window_size} seconds
  for each group with the same ['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p'].

  Args:
      df: The pandas DataFrame containing network flow data.
      window_size: Size of the window for calculating the average (default: 5000 seconds).

  Returns:
      A new DataFrame with the added aggregated feautre columns.
  """
  # Convert timestamp to datetime
  # df['ts'] = datetime.fromtimestamp(df['ts']) #assumes timestamps are in the local machine's timezone. not suggested  
  df['ts'] = pd.to_datetime(df['ts'], unit='s') 
  df = df.set_index('ts') 
  grp = ['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p']
  df[f'{agg_feature}_nunique_{window_size}'] = df.groupby(grp)[f'{agg_feature}'].transform(lambda x: x.rolling(f'{window_size}s', min_periods=1).apply(lambda x: x.unique().shape[0]))
  df[f'{agg_feature}_entropy_{window_size}'] = df.groupby(grp)[f'{agg_feature}'].transform(lambda x: x.rolling(f'{window_size}s', min_periods=1).apply(lambda x: entropy(x.value_counts()))) 
  return df.reset_index()

result_df = calculate_agg_feature_cat(df.copy(),agg_feature = 'local_orig')  # Apply function to a copy of df
print(result_df)


                              ts                 uid               id.orig_h  \
0  2024-02-12 17:10:23.364103168  C0LEGs2p93lnNEFB5f           192.168.0.168   
1  2024-02-12 17:10:23.364033024  CoIaps3LBPANWZX887           192.168.0.168   
2  2024-02-12 17:10:28.897382144  CllN3R2OE84qgP4Myl           192.168.0.168   
3  2024-02-12 17:10:23.682648832   CnAcZTvINoaJSPmC2           192.168.0.168   
4  2024-02-12 17:10:23.682689024   CrKvqoo2pkmR8IcKf  fe80::17:2915:d910:f37   
..                           ...                 ...                     ...   
60 2024-02-12 18:12:59.359659008  CmsBId1mQSH21Nn8Xf           192.168.0.168   
61 2024-02-12 18:12:59.394697984  CPrXFE2XI3xFN2x4x6           192.168.0.168   
62 2024-02-12 18:12:59.394788096   CKX629PWRVQsJH3Df           192.168.0.168   
63 2024-02-12 18:12:59.396743936  CGzmAv3TE4THf4pQ5d           192.168.0.168   
64 2024-02-12 18:13:01.153886976  CUN2d84adrzHQOxdJi           192.168.0.168   

    id.orig_p      id.resp_h  id.resp_p

In [500]:
result_df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,conn_state,...,resp_bytes,is_destination_broadcast,traffic_direction,duration_mean_5000,duration_min_5000,duration_max_5000,duration_std_5000,duration_var_5000,duration_cnt_5000,duration_sum_5000
0,2024-02-12 17:10:23.364103168,C0LEGs2p93lnNEFB5f,192.168.0.168,65363,192.168.0.1,53,udp,dns,0.019866,SF,...,0,0,internal,0.019866,0.019866,0.019866,0.0,0.0,1.0,0.019866
1,2024-02-12 17:10:23.364033024,CoIaps3LBPANWZX887,192.168.0.168,51657,192.168.0.1,53,udp,dns,0.012296,SF,...,0,0,internal,0.012296,0.012296,0.012296,0.0,0.0,1.0,0.012296
2,2024-02-12 17:10:28.897382144,CllN3R2OE84qgP4Myl,192.168.0.168,51400,142.250.80.74,443,tcp,other,0.016519,RSTR,...,0,0,outgoing,0.016519,0.016519,0.016519,0.0,0.0,1.0,0.016519
3,2024-02-12 17:10:23.682648832,CnAcZTvINoaJSPmC2,192.168.0.168,5353,224.0.0.251,5353,udp,dns,7.450594,S0,...,0,0,outgoing,7.450594,7.450594,7.450594,0.0,0.0,1.0,7.450594
4,2024-02-12 17:10:23.682689024,CrKvqoo2pkmR8IcKf,fe80::17:2915:d910:f37,5353,ff02::fb,5353,udp,dns,7.450569,S0,...,0,0,IPv6,7.450569,7.450569,7.450569,0.0,0.0,1.0,7.450569


In [495]:
fil = result_df[(result_df['id.orig_h'] == '192.168.0.168') & (result_df['id.orig_p'] == 51428)& (result_df['id.resp_h'] == '142.250.65.234') & (result_df['id.resp_p'] == 443)]

In [496]:
fil

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,conn_state,...,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,orig_bytes,resp_bytes,is_destination_broadcast,traffic_direction,local_orig_nunique_5000,local_orig_entropy_5000
31,2024-02-12 17:09:42.990166016,ClKxU23lRlw5hdtGFj,192.168.0.168,51428,142.250.65.234,443,tcp,ssl,0.141091,S1,...,10,1658,14,7626,0,0,0,outgoing,1.0,0.0
42,2024-02-12 18:12:59.069338112,CJRUNP37hqlOb5Poa5,192.168.0.168,51428,142.250.65.234,443,tcp,other,0.012353,RSTR,...,4,397,1,40,0,0,0,outgoing,1.0,0.0


In [295]:
(0.141091+0.012353)/2

0.076722

#### Full function 

In [539]:
id_feature = ["id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p"]
features = ["ts","uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p",
            "proto", "service", "duration", "conn_state", "local_orig","local_resp",
            "missed_bytes","history", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]
data_list = []
for line in json_data_file.splitlines():
    # log_entry is now a single json log from the file
    log_entry = json.loads(line.strip())
    
    # Check if each feature is present in the log_entry
    feature_values = [log_entry.get(feature, None) for feature in features]
    data_list.append(feature_values)

df = pd.DataFrame(data_list, columns=features)

In [541]:
from columns import Aggr_conn
def preprocess_json_conn_agg(json_batch):
    """
    This function receives a json batch from the main control flow of the train 
    functions. It should convert the conn.log of the json_batch to a numpy 2D array, apply necessary transformations,
    then return it. 

    Note: the input is only one unzipped json file. 
    """
    features = ["ts","uid", "id.orig_h", "id.orig_p", "id.resp_h", "id.resp_p",
            "proto", "service", "duration", "conn_state", "local_orig","local_resp",
            "missed_bytes","history", "orig_pkts", "orig_ip_bytes", "resp_pkts", "resp_ip_bytes"]
    #TODO: add features: duration, local_orig, local_resp 
    data_list = []
    for line in json_batch.splitlines():
        # log_entry is now a single json log from the file 
        log_entry = json.loads(line.strip())
        # data_list.append([log_entry[feature] for feature in features])
        # Check if each feature is present in the log_entry
        feature_values = [log_entry.get(feature, None) for feature in features]
        data_list.append(feature_values)

    #TODO: optimize the code via removing pandas
    df = pd.DataFrame(data_list, columns=features) 

    #fill Nans with 0s : duration, orig_bytes resp_bytes
    df = fill_na(df)  
    # create history, broadcast, traffic_direction variables
    df = create_history_variable(df)
    df = create_broadcast_variable(df)
    df = create_direction_variable(df)

    # one hot encode categorical variables
    column_name = ['conn_state', "proto", "traffic_direction" , "service"]
    df = one_hot_encode(df, column_name)

    # Convert the boolean values in columns "local_orig" and "local_resp" to 1 and 0s
    df['local_orig'] = df['local_orig'].astype(int)
    df['local_resp'] = df['local_resp'].astype(int)

    #Compute Aggregated Features 
    windows = [60,3600,7200] #seconds 
    grp = ['id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p']
    aggr_feature_num = ['duration', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes']
    aggr_feature_cat = ['local_orig', 'local_resp']
    for window in windows:
        for feature in aggr_feature_num:
            df = calculate_agg_feature_num(df, feature, window)
        for feature in aggr_feature_cat:
            df = calculate_agg_feature_cat(df, feature, window)
    cols = Aggr_conn
    # make sure the columns are the same 
    df = makedf_samecol(cols, df)        
    # Convert DataFrame to NumPy array
    np_arr = df.to_numpy(dtype=np.float32)
    return np_arr

In [542]:
preprocess_json_conn_agg(json_data_file)

array([[0.01986599, 1.        , 1.        , ..., 0.        , 1.        ,
        0.        ],
       [0.01229596, 1.        , 1.        , ..., 0.        , 1.        ,
        0.        ],
       [0.01651907, 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.3383119 , 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.343127  , 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.02273989, 1.        , 0.        , ..., 0.        , 1.        ,
        0.        ]], dtype=float32)

## dns

In [294]:
current_dir_path = '/usr/local/logs/2024-02-12'
if not os.path.islink(current_dir_path):
    # sub_dir is now any given historical data directory 
    logging.info(f"Checking {current_dir_path}")
    for file in os.listdir(current_dir_path):
        # file is now any given file in the historical data directory
        current_file_path = os.path.join(current_dir_path, file)
        if "dns." in file: #conn.
            # get the whole file in memory
            logging.info(f"Opening file {current_file_path}")
            json_data_file = ungzip(current_file_path)
            # print(current_file_path)
            # print(json_data_file)

In [295]:
import json 
for line in json_data_file.splitlines():
        # log_entry is now a single json log from the file
        log_entry = json.loads(line.strip())
        print(log_entry)

{'ts': 1707768003.869346, 'uid': 'CS9fzl4EIr1i9ibne5', 'id.orig_h': '10.19.235.169', 'id.orig_p': 65501, 'id.resp_h': '128.122.0.71', 'id.resp_p': 53, 'proto': 'udp', 'trans_id': 3613, 'rtt': 0.01270914077758789, 'query': 'guzzoni.apple.com', 'qclass': 1, 'qclass_name': 'C_INTERNET', 'qtype': 65, 'qtype_name': 'HTTPS', 'rcode': 0, 'rcode_name': 'NOERROR', 'AA': False, 'TC': False, 'RD': True, 'RA': True, 'Z': 0, 'answers': ['guzzoni-apple-com.v.aaplimg.com'], 'TTLs': [3321.0], 'rejected': False}
{'ts': 1707768003.869397, 'uid': 'C7KsDehrLNsfmgqo', 'id.orig_h': '10.19.235.169', 'id.orig_p': 53141, 'id.resp_h': '128.122.0.71', 'id.resp_p': 53, 'proto': 'udp', 'trans_id': 5769, 'rtt': 0.012659072875976562, 'query': 'guzzoni.apple.com', 'qclass': 1, 'qclass_name': 'C_INTERNET', 'qtype': 1, 'qtype_name': 'A', 'rcode': 0, 'rcode_name': 'NOERROR', 'AA': False, 'TC': False, 'RD': True, 'RA': True, 'Z': 0, 'answers': ['guzzoni-apple-com.v.aaplimg.com', '34.225.66.6'], 'TTLs': [3321.0, 100.0], '

In [296]:
#utils 
import ipaddress
def one_hot_encode(df, column_name):
    for col in column_name:
        if col in df.columns:
            df = pd.get_dummies(data=df, columns=[col])
    return df

def create_broadcast_variable(new_df):
    # create broadcast variable
    # can have more than one broadcast address
    #255 is the broadcast address for ipv4 
    if 'id.resp_h' in new_df.columns:
        new_df['is_destination_broadcast'] = new_df['id.resp_h'].apply(lambda x: 1 if "255" in x[-3:] else 0) 
    return new_df

def create_direction_variable(new_df):
    # #create traffic direction variable
    new_df['traffic_direction']        = new_df.apply(lambda x: get_traffic_direction(x['id.orig_h'], x['id.resp_h']), axis=1) 
    return new_df


def get_traffic_direction(source_ip, destination_ip):
    """
    Takes a source and destination IP address and returns the direction of the traffic.
    Please ensure the source and destination are correct as this is useless without the verification of the parameters.

    Parameters
    ----------
    source_ip: str
        Source IP address of the flow.
    destination_ip: str
        Destination IP address of the flow.
    
    Returns
    -------
    str: string indicating the direction. Can be 'internal', 'outgoing', 'incoming' or 'external'.
    """
    src_ip = ipaddress.ip_address(source_ip) 
    dest_ip = ipaddress.ip_address(destination_ip) 
    if src_ip.version == 6 or dest_ip.version ==6:
        return "IPv6"
    
    if is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "internal"
    elif is_private_ip(source_ip) and not is_private_ip(destination_ip):
        return "outgoing"
    elif not is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "incoming"
    else:
        return "external"

# def is_private_ip(ip_str):
#     """
#     Takes an IP string and returns whether the IP is private or not per RFC 1918.

#     Parameters
#     ----------
#     ip_str: str
#         String of an IP address.

#     Returns
#     -------
#     bool: a bool of whether or not the IP is private. 
#     """
#     octets = [int(x) for x in ip_str.split(".")]
#     if octets[0] == 10 \
#             or (octets[0] == 172 and 16 <= octets[1] <= 31) \
#             or (octets[0] == 192 and octets[1] == 168):
#         return True
#     else:
#         return False
    


def is_private_ip(ip_str):
    """
    Takes an IP string and returns whether the IP is private or not per RFC 1918.

    Parameters
    ----------
    ip_str: str
        String of an IP address.

    Returns
    -------
    bool: a bool of whether or not the IP is private.
    """
    try:
        ip = ipaddress.ip_address(ip_str)
        if ip.version == 4:
            return ip.is_private
        else:
            return False  # Ignore IPv6 addresses
    except ValueError:
        return False  # Invalid IP address format

def makedf_samecol(cols, new_df):
    #Create these columns if they are not present in the original df and fill them with 0s. 
    # Ensure that all the specified columns are present even if they are not present in the original df. 

    for col in cols:
        if col not in new_df.columns:
            new_df[col] = 0
    return new_df[cols]

#conn + Four other logs: dns, http, ssl, ssh 
#for dns: proto (str), rtt(float), qclass(num as type), qtype(str), rcode(numeric 0-9), AA (str, true or false), TC(str, true or false), RD(str, true or false), RA(str, true or false), Z, rejected(str, true or false) 
TODO: check what is the value of Z, and do we need domain name. check what might not be in the logs (so far: rcode,rtt )
#for http: trans_depth, method, 


In [297]:
features = ['id.orig_h', "id.resp_h", "proto", "rtt","qclass_name", "qtype_name","rcode_name",
                "AA","TC","RD","RA", "rejected"]
        
data_list = []
for line in json_data_file.splitlines():
    # log_entry is now a single json log from the file
    log_entry = json.loads(line.strip())
    
    # Check if each feature is present in the log_entry
    feature_values = [log_entry.get(feature, None) for feature in features]
    data_list.append(feature_values)

df = pd.DataFrame(data_list, columns=features)

In [298]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'proto', 'rtt', 'qclass_name', 'qtype_name',
       'rcode_name', 'AA', 'TC', 'RD', 'RA', 'rejected'],
      dtype='object')

In [299]:
df['rcode_name'].value_counts()
#there are total of 1984 rows 
# NOERROR     1860
# NXDOMAIN      12

NOERROR     1860
NXDOMAIN      12
Name: rcode_name, dtype: int64

In [300]:
#This code checks for null values in each feature
has_null = []
for feature in df.columns:
    null_count = df[feature].isnull().sum()
    print(f"Null count for {feature}: {null_count}")    
    if null_count:
        has_null.append(feature)

# Create a variable for each feature that contains null, with the column name "has_null_featurename"
for feature in has_null: 
    df[f'has_{feature}'] = df[feature].notnull().astype(int)


Null count for id.orig_h: 0
Null count for id.resp_h: 0
Null count for proto: 0
Null count for rtt: 736
Null count for qclass_name: 73
Null count for qtype_name: 73
Null count for rcode_name: 112
Null count for AA: 0
Null count for TC: 0
Null count for RD: 0
Null count for RA: 0
Null count for rejected: 0


In [301]:
has_null 

['rtt', 'qclass_name', 'qtype_name', 'rcode_name']

In [302]:
#create broadcast and direction variables
df = create_broadcast_variable(df)
df = create_direction_variable(df)

In [303]:
print(df['traffic_direction'].shape)
print(df['traffic_direction'].value_counts())
print(df['traffic_direction'].isnull().sum())

(1984,)
outgoing    1682
IPv6         302
Name: traffic_direction, dtype: int64
0


In [305]:
#one hot encode qtype, qclass, rcode_name
column_name = ['proto','qtype_name','qclass_name','rcode_name','traffic_direction']
df = one_hot_encode(df, column_name)

In [306]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'rtt', 'AA', 'TC', 'RD', 'RA', 'rejected',
       'has_rtt', 'has_qclass_name', 'has_qtype_name', 'has_rcode_name',
       'is_destination_broadcast', 'proto_udp', 'qtype_name_*', 'qtype_name_A',
       'qtype_name_AAAA', 'qtype_name_HTTPS', 'qtype_name_PTR',
       'qclass_name_C_INTERNET', 'qclass_name_qclass-32769',
       'rcode_name_NOERROR', 'rcode_name_NXDOMAIN', 'traffic_direction_IPv6',
       'traffic_direction_outgoing'],
      dtype='object')

In [307]:
#encode boolean features 
boolean_to_convert = ['AA', 'TC', 'RD', 'RA', 'rejected']
df[boolean_to_convert] = df[boolean_to_convert].astype(int)


In [308]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'rtt', 'AA', 'TC', 'RD', 'RA', 'rejected',
       'has_rtt', 'has_qclass_name', 'has_qtype_name', 'has_rcode_name',
       'is_destination_broadcast', 'proto_udp', 'qtype_name_*', 'qtype_name_A',
       'qtype_name_AAAA', 'qtype_name_HTTPS', 'qtype_name_PTR',
       'qclass_name_C_INTERNET', 'qclass_name_qclass-32769',
       'rcode_name_NOERROR', 'rcode_name_NXDOMAIN', 'traffic_direction_IPv6',
       'traffic_direction_outgoing'],
      dtype='object')

In [309]:
for feature in df.columns:
    null_count = df[feature].isnull().sum()
    print(f"Null count for {feature}: {null_count}")   

Null count for id.orig_h: 0
Null count for id.resp_h: 0
Null count for rtt: 736
Null count for AA: 0
Null count for TC: 0
Null count for RD: 0
Null count for RA: 0
Null count for rejected: 0
Null count for has_rtt: 0
Null count for has_qclass_name: 0
Null count for has_qtype_name: 0
Null count for has_rcode_name: 0
Null count for is_destination_broadcast: 0
Null count for proto_udp: 0
Null count for qtype_name_*: 0
Null count for qtype_name_A: 0
Null count for qtype_name_AAAA: 0
Null count for qtype_name_HTTPS: 0
Null count for qtype_name_PTR: 0
Null count for qclass_name_C_INTERNET: 0
Null count for qclass_name_qclass-32769: 0
Null count for rcode_name_NOERROR: 0
Null count for rcode_name_NXDOMAIN: 0
Null count for traffic_direction_IPv6: 0
Null count for traffic_direction_outgoing: 0


In [310]:
#fillna with 0s:rtt
columns_to_fill_with_zeros = ['rtt']
df[columns_to_fill_with_zeros] = df[columns_to_fill_with_zeros].fillna(0)

In [311]:
for feature in df.columns:
    null_count = df[feature].isnull().sum()
    print(f"Null count for {feature}: {null_count}")   

Null count for id.orig_h: 0
Null count for id.resp_h: 0
Null count for rtt: 0
Null count for AA: 0
Null count for TC: 0
Null count for RD: 0
Null count for RA: 0
Null count for rejected: 0
Null count for has_rtt: 0
Null count for has_qclass_name: 0
Null count for has_qtype_name: 0
Null count for has_rcode_name: 0
Null count for is_destination_broadcast: 0
Null count for proto_udp: 0
Null count for qtype_name_*: 0
Null count for qtype_name_A: 0
Null count for qtype_name_AAAA: 0
Null count for qtype_name_HTTPS: 0
Null count for qtype_name_PTR: 0
Null count for qclass_name_C_INTERNET: 0
Null count for qclass_name_qclass-32769: 0
Null count for rcode_name_NOERROR: 0
Null count for rcode_name_NXDOMAIN: 0
Null count for traffic_direction_IPv6: 0
Null count for traffic_direction_outgoing: 0


In [ ]:
#TODO: to be confirmed once EDA is done
dns_cols = ['rtt', 'AA', 'TC', 'RD', 'RA', 'rejected',
       'has_rtt', 'has_qclass_name', 'has_qtype_name', 'has_rcode_name',
       'is_destination_broadcast', 
       'proto_tcp', 'proto_udp',
       'qtype_name_*', 'qtype_name_A',
       'qtype_name_AAAA', 'qtype_name_HTTPS', 'qtype_name_PTR',
       'qclass_name_C_INTERNET', 'qclass_name_qclass-32769',
       'rcode_name_NOERROR', 'rcode_name_NXDOMAIN', 
       'traffic_direction_IPv6',
       'traffic_direction_external','traffic_direction_incoming', 
        'traffic_direction_internal','traffic_direction_outgoing']


## http

In [338]:
current_dir_path = '/usr/local/logs/2024-02-12'
if not os.path.islink(current_dir_path):
    # sub_dir is now any given historical data directory 
    logging.info(f"Checking {current_dir_path}")
    for file in os.listdir(current_dir_path):
        # file is now any given file in the historical data directory
        current_file_path = os.path.join(current_dir_path, file)
        if "http." in file: #conn.
            # get the whole file in memory
            logging.info(f"Opening file {current_file_path}")
            json_data_file = ungzip(current_file_path)
            # print(current_file_path)
            # print(json_data_file)

In [339]:
import json 
for line in json_data_file.splitlines():
        # log_entry is now a single json log from the file
        log_entry = json.loads(line.strip())
        print(log_entry)

{'ts': 1707721209.155077, 'uid': 'CGpf9R1APvMKSdDIn2', 'id.orig_h': '192.168.0.109', 'id.orig_p': 57562, 'id.resp_h': '192.168.0.168', 'id.resp_p': 7000, 'trans_depth': 1, 'method': 'GET', 'uri': '/info?txtAirPlay&txtRAOP RTSP/1.', 'version': '0.9', 'request_body_len': 0, 'response_body_len': 1754, 'status_code': 0, 'status_msg': '<empty>', 'tags': [], 'resp_fuids': ['FuhWEF48bHYDzu07H7'], 'resp_mime_types': ['text/plain']}
{'ts': 1707721241.169723, 'uid': 'Cyl3PW2wswIPW1tjY7', 'id.orig_h': '192.168.0.168', 'id.orig_p': 62770, 'id.resp_h': '17.253.3.220', 'id.resp_p': 80, 'trans_depth': 1, 'method': 'CONNECT', 'host': 'proxy-safebrowsing.googleapis.com', 'uri': 'proxy-safebrowsing.googleapis.com:443', 'version': '1.1', 'request_body_len': 0, 'response_body_len': 0, 'status_code': 200, 'status_msg': 'OK', 'tags': [], 'proxied': ['PROXY-CONNECTION -> keep-alive']}
{'ts': 1707723317.563444, 'uid': 'CNKryg21vxMXbO22ud', 'id.orig_h': '192.168.0.168', 'id.orig_p': 62836, 'id.resp_h': '17.253

In [340]:
#utils 
import ipaddress
def one_hot_encode(df, column_name):
    for col in column_name:
        if col in df.columns:
            df = pd.get_dummies(data=df, columns=[col])
    return df

def create_broadcast_variable(new_df):
    # create broadcast variable
    # can have more than one broadcast address
    #255 is the broadcast address for ipv4 
    if 'id.resp_h' in new_df.columns:
        new_df['is_destination_broadcast'] = new_df['id.resp_h'].apply(lambda x: 1 if "255" in x[-3:] else 0) 
    return new_df

def create_direction_variable(new_df):
    # #create traffic direction variable
    new_df['traffic_direction']        = new_df.apply(lambda x: get_traffic_direction(x['id.orig_h'], x['id.resp_h']), axis=1) 
    return new_df


def get_traffic_direction(source_ip, destination_ip):
    """
    Takes a source and destination IP address and returns the direction of the traffic.
    Please ensure the source and destination are correct as this is useless without the verification of the parameters.

    Parameters
    ----------
    source_ip: str
        Source IP address of the flow.
    destination_ip: str
        Destination IP address of the flow.
    
    Returns
    -------
    str: string indicating the direction. Can be 'internal', 'outgoing', 'incoming' or 'external'.
    """
    src_ip = ipaddress.ip_address(source_ip) 
    dest_ip = ipaddress.ip_address(destination_ip) 
    if src_ip.version == 6 or dest_ip.version ==6:
        return "IPv6"
    
    if is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "internal"
    elif is_private_ip(source_ip) and not is_private_ip(destination_ip):
        return "outgoing"
    elif not is_private_ip(source_ip) and is_private_ip(destination_ip):
        return "incoming"
    else:
        return "external"
    

def is_private_ip(ip_str):
    """
    Takes an IP string and returns whether the IP is private or not per RFC 1918.

    Parameters
    ----------
    ip_str: str
        String of an IP address.

    Returns
    -------
    bool: a bool of whether or not the IP is private.
    """
    try:
        ip = ipaddress.ip_address(ip_str)
        if ip.version == 4:
            return ip.is_private
        else:
            return False  # Ignore IPv6 addresses
    except ValueError:
        return False  # Invalid IP address format

def makedf_samecol(cols, new_df):
    #Create these columns if they are not present in the original df and fill them with 0s. 
    # Ensure that all the specified columns are present even if they are not present in the original df. 

    for col in cols:
        if col not in new_df.columns:
            new_df[col] = 0
    return new_df[cols]

In [321]:
features = ['id.orig_h', 'id.resp_h','trans_depth','method','host','version','request_body_len','response_body_len','status_code']
        
data_list = []
for line in json_data_file.splitlines():
    # log_entry is now a single json log from the file
    log_entry = json.loads(line.strip())
    
    # Check if each feature is present in the log_entry
    feature_values = [log_entry.get(feature, None) for feature in features]
    data_list.append(feature_values)

df = pd.DataFrame(data_list, columns=features)

In [341]:
df.head()

,id.orig_h,id.resp_h,trans_depth,host,version,request_body_len,response_body_len,has_host,is_destination_broadcast,method_CONNECT,method_GET,status_code_0,status_code_200,traffic_direction_internal,traffic_direction_outgoing
0,192.168.0.109,192.168.0.168,1,None,0.9,0,1754,0,0,0,1,1,0,1,0
1,192.168.0.168,17.253.3.220,1,proxy-safebrowsing.googleapis.com,1.1,0,0,1,0,1,0,0,1,0,1
2,192.168.0.168,17.253.3.217,1,proxy-safebrowsing.googleapis.com,1.1,0,0,1,0,1,0,0,1,0,1
3,192.168.0.168,192.229.211.108,1,ocsp.digicert.com,1.1,0,471,1,0,0,1,0,1,0,1
4,192.168.0.168,192.229.211.108,1,ocsp.digicert.com,1.1,0,313,1,0,0,1,0,1,0,1


In [342]:
#This code checks for null values in each feature
has_null = []
for feature in df.columns:
    null_count = df[feature].isnull().sum()
    print(f"Null count for {feature}: {null_count}")    
    if null_count:
        has_null.append(feature)

print("\nhas_null",has_null) # has_null ['host'] 

# Create a variable for each feature that contains null, with the column name "has_null_featurename"
for feature in has_null: 
    df[f'has_{feature}'] = df[feature].notnull().astype(int)


Null count for id.orig_h: 0
Null count for id.resp_h: 0
Null count for trans_depth: 0
Null count for host: 1
Null count for version: 0
Null count for request_body_len: 0
Null count for response_body_len: 0
Null count for has_host: 0
Null count for is_destination_broadcast: 0
Null count for method_CONNECT: 0
Null count for method_GET: 0
Null count for status_code_0: 0
Null count for status_code_200: 0
Null count for traffic_direction_internal: 0
Null count for traffic_direction_outgoing: 0

has_null ['host']


In [343]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'trans_depth', 'host', 'version',
       'request_body_len', 'response_body_len', 'has_host',
       'is_destination_broadcast', 'method_CONNECT', 'method_GET',
       'status_code_0', 'status_code_200', 'traffic_direction_internal',
       'traffic_direction_outgoing'],
      dtype='object')

In [ ]:
#identify null
#create has null
# create broadcast, traffic_direction variables
# one hot encode categorical variables: 'method','status_code','traffic_direction'
#fillna with 0s: len?? (no na in len)
#same columns  (no 'host)

In [344]:
df = create_broadcast_variable(df)
df = create_direction_variable(df)

In [345]:
column_name = ['version','method','status_code','traffic_direction']
df = one_hot_encode(df, column_name)

In [346]:
df.head()

,id.orig_h,id.resp_h,trans_depth,host,request_body_len,response_body_len,has_host,is_destination_broadcast,method_CONNECT,method_GET,status_code_0,status_code_200,traffic_direction_internal,traffic_direction_outgoing,version_0.9,version_1.1,traffic_direction_internal,traffic_direction_outgoing
0,192.168.0.109,192.168.0.168,1,None,0,1754,0,0,0,1,1,0,1,0,1,0,1,0
1,192.168.0.168,17.253.3.220,1,proxy-safebrowsing.googleapis.com,0,0,1,0,1,0,0,1,0,1,0,1,0,1
2,192.168.0.168,17.253.3.217,1,proxy-safebrowsing.googleapis.com,0,0,1,0,1,0,0,1,0,1,0,1,0,1
3,192.168.0.168,192.229.211.108,1,ocsp.digicert.com,0,471,1,0,0,1,0,1,0,1,0,1,0,1
4,192.168.0.168,192.229.211.108,1,ocsp.digicert.com,0,313,1,0,0,1,0,1,0,1,0,1,0,1


In [347]:
df.columns

Index(['id.orig_h', 'id.resp_h', 'trans_depth', 'host', 'request_body_len',
       'response_body_len', 'has_host', 'is_destination_broadcast',
       'method_CONNECT', 'method_GET', 'status_code_0', 'status_code_200',
       'traffic_direction_internal', 'traffic_direction_outgoing',
       'version_0.9', 'version_1.1', 'traffic_direction_internal',
       'traffic_direction_outgoing'],
      dtype='object')

In [348]:
df['has_host']

0    0
1    1
2    1
3    1
4    1
5    1
6    1
7    1
Name: has_host, dtype: int64

In [ ]:
#TODO: to be confirmed once EDA is done
http_cols = ['trans_depth', 'request_body_len',
       'response_body_len', 'has_host', 'is_destination_broadcast',
       'method_CONNECT', 'method_GET', 
       'status_code_0', 'status_code_200',
       'version_0.9', 'version_1.1',
       'traffic_direction_IPv6',
       'traffic_direction_internal', 'traffic_direction_outgoing',
       'traffic_direction_internal','traffic_direction_outgoing']
